In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
index_to_save = 5

<IPython.core.display.Javascript object>

In [4]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [5]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_iv",
    "Features": "Chemical + Mineralogical + Properties CS Less",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [6]:
df = pd.read_csv("../../../../../data/processed/partner_iv/cem_b.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [7]:
df_copy = df.drop(
    [
        # Properties
        "CS2",  # 2-day Compressive Strength
    ],
    axis=1,
).copy()

<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [8]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [9]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.863 (0.032)
MAE: -1.509 (0.029)
MAPE: -0.029 (0.001)
R2: 0.494 (0.021)


******
[TEST]
******
RMSE: -2.160 (0.158)
MAE: -1.739 (0.140)
MAPE: -0.033 (0.003)
R2: 0.297 (0.119)




<IPython.core.display.Javascript object>

In [10]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   

  Cross Validation RMSE Test           MAE Test           MAPE Test            \
                        mean      std      mean       std      mean       std   
0   Repeated KFold  2.160023  0.15766  1.738824  0.139764  0.033464  0.002664   

    R2 Test           
       mean      std  
0  0.297328  0.11912

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [11]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.024 (0.230)
MAE: -0.811 (0.207)
MAPE: -0.016 (0.004)
R2: 0.822 (0.080)


******
[TEST]
******
RMSE: -3.951 (1.246)
MAE: -3.193 (1.129)
MAPE: -0.061 (0.020)
R2: -5.441 (5.544)




<IPython.core.display.Javascript object>

In [12]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.950650  1.246071  3.193366  1.129188   
1              Repeated KFold  2.160023  0.157660  1.738824  0.139764   

  MAPE Test             R2 Test           
       mean       std      mean      std  
0  0.061373  0.020135 -5.441167  5.54423  
1  0.033464  0.002664  0.297328  0.11912

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [13]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.588 (0.249)
MAE: -1.268 (0.207)
MAPE: -0.024 (0.004)
R2: 0.584 (0.083)


******
[TEST]
******
RMSE: -2.742 (0.370)
MAE: -2.277 (0.320)
MAPE: -0.044 (0.007)
R2: -0.358 (0.245)




<IPython.core.display.Javascript object>

In [14]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.950650  1.246071  3.193366  1.129188   
1              Repeated KFold  2.160023  0.157660  1.738824  0.139764   
2           Time Series Split  2.741629  0.370098  2.277172  0.319579   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.061373  0.020135 -5.441167  5.544230  
1  0.033464  0.002664  0.297328  0.119120  
2  0.043889  0.007046 -0.358400  0.245414

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [15]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 1.724 (0.000)
MAE: 1.406 (0.000)
MAPE: 0.027 (0.000)
R2: 0.581 (0.000)


******
[TEST]
******
RMSE: 2.762 (0.000)
MAE: 2.239 (0.000)
MAPE: 0.044 (0.000)
R2: -0.362 (0.000)




<IPython.core.display.Javascript object>

In [16]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   
3  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.950650  1.246071  3.193366  1.129188   
1           Out of time Split  2.761539  0.000000  2.239382  0.000000   
2              Repeated KFold  2.160023  0.157660  1.738824  0.139764   
3           Time Series Split  2.741629  0.370098  2.277172  0.319579   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.061373  0.020135 -5.441167  5.544230  
1  0.043638  0.000000 -0.361644  0.000000  
2  0.033464  0.002664  0.297328  0.119120  
3  0.043889  0.007046 -0.358400  0.245414

<IPython.core.display.Javascript object>

In [17]:
pipeline.named_steps["estimator"].coef_

array([ 4.63355912e-02, -1.06932651e-01, -8.14898861e-01, -1.30701702e-01,
        3.69340722e-01,  1.09419435e-01, -6.71621909e-02, -1.67371923e+00,
       -1.70906042e-01, -7.08190687e+01, -6.37270085e+01,  2.94026962e+01,
        5.77906948e-01, -2.82192302e-02, -1.02566898e+01,  2.14161717e+00,
        4.73204389e+00, -7.29566504e+00, -3.28419160e+00,  4.68751028e+01,
        3.98738190e+01, -5.08203075e+01, -7.17575018e-01, -9.15217197e-01,
        1.04769583e+00,  9.34780894e-01,  1.52686895e+00,  4.38032253e+00,
        4.19371185e+00,  5.37244383e+00, -1.53749273e+01, -6.35665244e+00,
        7.41396037e+00])

<IPython.core.display.Javascript object>

In [18]:
pipeline.named_steps["estimator"].intercept_

52.24069767441855

<IPython.core.display.Javascript object>

In [19]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [20]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['SO3', 'Cl-', 'Blaine', 'ph2oimm', 'Initial Setting Time', 'Soundness',
       'flow', 'residuo 24 micron', 'R_wp', 'Alite_M3 C3S M3',
       'Alite_M1 C3S M1', 'Alite_Sum C3S tot', 'Ratio_M1 (rapporto M1/M3)',
       'C3S_CS (taglia dei cristalliti C3S)', 'Belite_beta', 'C3A cub',
       'C3A_ortho', 'C3A tot', 'C4AF', 'CaO', 'Ca(OH)2', 'Calce libera',
       'Periclasio (MgO)', 'Quartz', 'K2SO4', 'Langbeinite – MgK2(SO4)2',
       'Aphthitalite – (K,Na)3(SO4)2', 'Gesso', 'Emiidrato', 'Anidrite',
       'Calcite – CaCO3', 'SO3_XRD', 'CO2_XRD'],
      dtype='object')

<IPython.core.display.Javascript object>

In [21]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(33,)

<IPython.core.display.Javascript object>

In [22]:
x.columns.shape

(33,)

<IPython.core.display.Javascript object>

In [23]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [24]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
CaO,46.875103
Ca(OH)2,39.873819
Alite_Sum C3S tot,29.402696
CO2_XRD,7.413960
Anidrite,5.372444
C3A_ortho,4.732044
Gesso,4.380323
Emiidrato,4.193712
C3A cub,2.141617
"Aphthitalite – (K,Na)3(SO4)2",1.526869


<IPython.core.display.Javascript object>

In [25]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   
3  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  3.950650  1.246071   
1                                {"Test Size": 0.2}  2.761539  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  2.160023  0.157660   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  2.741629  0.370098   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  3.193366  1.129188  0.061373  0.020135 -5.441167  5.544230  
1  2.239382  0.000000  0.043638  0.000000 -0.361644  0.000000  
2  1.738824  0.139764  0.033464  0.002664  0.297328  0.119120  
3  2.277172  0.319579  0.043889  0.007046 -0.358400  0.245414

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [26]:
path = "../../../../../reports/results/local_models/partner_iv/cem_b/full/"
filename = f"linear_regression_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [27]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_iv/cem_b/grouped/"
filename = f"linear_regression_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [28]:
df_results_to_save

Category Company Data Shape Timesteps  \
                                              
0  Local Model   partner_iv  (323, 33)       NaN   
1  Local Model   partner_iv  (323, 33)       NaN   
2  Local Model   partner_iv  (323, 33)       NaN   
3  Local Model   partner_iv  (323, 33)       NaN   

                                        Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   
3  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   1.024126  0.230129  ...   
1                                {"Test Size": 0.2}   1.723826  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   1.863372  0.032401  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   1.587579  0.248662  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.821853  0.080346  3.950650  1.246071  3.193366  1.129188  0.061373   
1  0.580525  0.000000  2.761539  0.000000  2.239382  0.000000  0.043638   
2  0.493570  0.020744  2.160023  0.157660  1.738824  0.139764  0.033464   
3  0.583532  0.083083  2.741629  0.370098  2.277172  0.319579  0.043889   

              R2 Test            
        std      mean       std  
0  0.020135 -5.441167  5.544230  
1  0.000000 -0.361644  0.000000  
2  0.002664  0.297328  0.119120  
3  0.007046 -0.358400  0.245414  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>